<a href="https://colab.research.google.com/github/kerryback/ClassicTests/blob/main/portfolios/constrained_optimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

try :
    from cvxopt import matrix
except :
    !pip install cvxopt
    from cvxopt import matrix
    
from cvxopt.solvers import qp as Solver
from cvxopt.solvers import options as SolverOptions
SolverOptions['show_progress'] = False
    
def optimal_noshorts_noleverage(rprem,cov,raver) :
    num = len(rprem)
    Q = matrix(raver*cov)                                    # quadratic form that penalizes variance
    p = matrix(-rprem,(num,1))                               # want to maximize wtd avg of risk premia
    G = matrix(np.vstack([-np.identity(num),np.ones(num)]))  # imposes nonnegativity and weights sum to <=1
    h = matrix(np.vstack([np.zeros(num).reshape(num,1),1]))  # right-hand side of inequality constraints
    sol = Solver(Q,p,G,h)
    return sol['x'] if sol['status']=='optimal' else None

# example data

means = np.array([0.10,0.15,0.20])  # expected returns
sds = np.array([0.15,0.25,0.30])    # standard deviations
c12 = 0.2                        # correlations (1 with 2, 1 with 3, and 2 with 3)
c13 = 0.5
c23 = 0.3
rf = 0.02                           # risk-free rate
raver = 4                           # risk aversion

# solution

corr = np.array([[1,c12,c13],[c12,1,c23],[c13,c23,1]])
cov = np.diag(sds) @ corr @ np.diag(sds)
rprem = means - rf
sol = optimal_noshorts_noleverage(rprem,cov,raver)
list(sol)

[0.37360672382078003, 0.3291654704849601, 0.2972224939294642]